# Omnigenity Optimization

This tutorial demonstrates how to optimize for omnigenity in DESC. It will go through an example using omnigenity with poloidally closed contours of magnetic field strength (OP), but the method is capable of optimizing for any general omnigenous magnetic fields as explained in [Dudt et al. (2024)](https://arxiv.org/abs/2305.08026).

In [ ]:
import sys
import os

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../../../"))

If you have access to a GPU, uncomment the following two lines. 

In [ ]:
# from desc import set_device
# set_device("gpu")

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["font.size"] = 12

import numpy as np

from desc.equilibrium import Equilibrium
from desc.geometry import FourierRZToroidalSurface
from desc.grid import LinearGrid
from desc.magnetic_fields import OmnigenousField
from desc.objectives import (
    AspectRatio,
    CurrentDensity,
    FixCurrent,
    FixOmni,
    FixPressure,
    FixPsi,
    FixWell,
    GenericObjective,
    ObjectiveFunction,
    Omnigenity,
    StraightBmaxContour,
    get_fixed_boundary_constraints,
)
from desc.optimize import Optimizer
from desc.plotting import plot_boozer_surface, plot_boundaries, plot_omnigenous_field

As an initial guess for the optimization, we will start with a boundary shape generated by an analytic model for (very approximate) quasi-poloidal symmetry (QP). 
In this example, we will seek a vacuum magnetic field with two field periods ($N_{FP}=2$), an aspect ratio of $R_0/a\leq10$, and a mirror ratio on axis of $\Delta=0.2$. 


In [ ]:
surf = FourierRZToroidalSurface.from_qp_model(
    major_radius=1,
    aspect_ratio=10,
    elongation=3,
    mirror_ratio=0.2,
    torsion=0.1,
    NFP=2,
    sym=True,
)
# this value of Psi gives an average |B| on axis of about 1 T
# the Equilibrium class defaults to vacuum pressure and current profiles
eq = Equilibrium(Psi=3e-2, M=4, N=4, surface=surf)

Now that the equilibrium is initialized, we need to solve the fixed-boundary vacuum equilibrium: 

In [ ]:
constraints = get_fixed_boundary_constraints(eq)
eq, _ = eq.solve(objective="vacuum", constraints=constraints, verbose=3)

Let us make a copy of this initial equilibrium so that we can compare our final solution to it later and see how well the omnigenity optimization worked. 
By plotting the $|B|$ contours in Boozer coordinates, we can see that this equilibrium is already somewhat close to being omnigenous with poloidal contours, but far from perfect. 

In [ ]:
eq0 = eq.copy()
plot_boozer_surface(eq0, fieldlines=4)  # defaults to the rho=1 surface

In order to optimize the equilibrium for omnigenity, we need to create a target omnigenous magnetic field. 
The `OmnigenousField` class has two attributes that represent parameters in the omnigenous magnetic field model: 
- `B_lm` specifies the shape of the "magnetic well" on each flux surface. 
- `x_lmn` specifies how the well shape varies along different field lines. 

We need to specify the resolution of the parameter space and the helicity of the field. 
The helicity is given by the tuple of integers $(M, N)$, and is set to $(0, N_{FP})$ for omnigenity with poloidal contours in this example. 
The typical case for helical contours would be $(1, N_{FP})$, and for toroidal contours would be $(1, 0)$. 

We provide initial values for the well shape parameters `B_lm` so that we can set the mirror ratio. 
The total number of parameters is `B_lm.size = M_well * (L_well + 1)`. 
In this example, the well shape on each surface is represented by three spline knots, and there is $\mathcal{O}(\rho)$ variation across the flux surfaces. 
Here we set only the constant terms of the Chebyshev polynomials such that the initial target field has the same magnetic well from $B_{\mathrm{min}}=0.8\mathrm{~T}$ to $B_{\mathrm{max}}=1.2\mathrm{~T}$ on each surface. 

The `x_lmn` parameters are left to their default values of 0. 

In [ ]:
field = OmnigenousField(
    L_well=1,  # radial resolution of B_lm parameters
    M_well=3,  # number of spline knots on each flux surface
    L_omni=1,  # radial resolution of x_lmn parameters
    M_omni=1,  # poloidal resolution of x_lmn parameters
    N_omni=1,  # toroidal resolution of x_lmn parameters
    NFP=eq.NFP,  # number of field periods; should always be equal to Equilibrium.NFP
    helicity=(0, eq.NFP),  # helicity for poloidally closed |B| contours
    B_lm=np.array([0.8, 1.0, 1.2, 0, 0, 0]),  # magnetic well shape parameters
)

Next we create the objective function for the optimization. 
We include one objective to keep the major radius at $R_0=1\textrm{~m}$, and another to keep the aspect ratio at $R_0/a\leq10$. 
The elongation will be unconstrained, but that is another common objective we could choose to include. 

We will also target omnigenity on two flux surfaces: $\rho=0.5$ and $\rho=1$. 
The `Omnigenity` objective class requires two different computational grids: 
- `grid_eq` is the grid used to compute the Boozer transform. 
- `grid_field` is the grid corresponding to $(\rho,\eta,\alpha)$ coordinates where the omnigenity residuals are minimized. 

A separate `Omnigenity` objective is required for each flux surface, but they all reference the same `Equilibrium` and `OmnigenousField`. Make sure both grids for each objective are at the desired surface and have `sym=False`! 

In [29]:
grid_eq_half = LinearGrid(rho=0.5, M=4 * eq.M, N=4 * eq.N, NFP=eq.NFP, sym=False)
grid_eq_lcfs = LinearGrid(rho=1.0, M=4 * eq.M, N=4 * eq.N, NFP=eq.NFP, sym=False)

grid_field_half = LinearGrid(rho=0.5, theta=16, zeta=8, NFP=field.NFP, sym=False)
grid_field_lcfs = LinearGrid(rho=1.0, theta=16, zeta=8, NFP=field.NFP, sym=False)

objective = ObjectiveFunction(
    (
        # target major radius of R0=1 m
        GenericObjective("R0", eq=eq, target=1.0, name="major radius"),
        # target aspect ratio R0/a<=10
        AspectRatio(eq=eq, bounds=(0, 10)),
        # omnigenity on the rho=0.5 surface
        Omnigenity(
            field=field,
            eq=eq,
            grid_eq=grid_eq_half,
            grid_field=grid_field_half,
            well_weight=1,
        ),
        # omnigenity on the rho=1.0 surface
        Omnigenity(
            field=field,
            eq=eq,
            grid_eq=grid_eq_lcfs,
            grid_field=grid_field_lcfs,
            well_weight=2,
        ),
    )
)

Next we set the optimization constraints. 
The `CurrentDensity`, `FixPressure`, `FixCurrent`, and `FixPsi` objectives ensure that we maintain a good vacuum equilibrium during the optimization. 

We also include three additional constraints that are unique to the omnigenity optimization: 
- A perfect omnigenous magnetic field must have a straight $B_{\mathrm{max}}$ contour in Boozer coordinates, and this is accomplished with the `StraightBmaxContour` objective. 
- The `FixOmni` objective is used to fix values of the `field.x_lmn` parameters. 
In this OP example our $B_{\mathrm{max}}$ contour is located at $\zeta_B=0$, and this constraint is used to ensure that the $B_{\mathrm{min}}$ contour is located at $\zeta_B=\pi/N_{FP}$ on average. 
This constraint should always be true for stellarator symmetry. 
- The `FixWell` objective is used to fix values of the `field.B_lm` parameters. 
Here we use it to fix the values of $B_{\mathrm{min}}$ and $B_{\mathrm{max}}$ on axis to contrain the mirror ratio. 
The shape of the magnetic well will still have one degree of freedom on the magnetic axis, and the mirror ratio is unconstrained on other flux surfaces. 

In [ ]:
constraints = (
    CurrentDensity(eq=eq),  # vacuum equilibrium force balance
    FixPressure(eq=eq),  # fix vacuum pressure profile
    FixCurrent(eq=eq),  # fix vacuum current profile
    FixPsi(eq=eq),  # fix total toroidal magnetic flux
    # ensure the B_max contour is straight in Boozer coordinates
    StraightBmaxContour(field=field),
    # ensure the average B_min contour is at zeta_B=pi/NFP
    FixOmni(field=field, indices=np.where(field.omni_basis.modes[:, 1] == 0)[0]),
    # fix the mirror ratio on the magnetic axis
    FixWell(field=field, indices=[0, field.M_well - 1]),
)

Finally we are ready to run the optimization! 
We use a least-squares augmented Lagrangian optimizer, which is the recommended algorithm for nonlinear constrained optimizations that optimize multiple "things" (in this example, the equilibrium `eq` and the omnigenous field `field`). 

In [ ]:
optimizer = Optimizer("lsq-auglag")
(eq, field), _ = optimizer.optimize((eq, field), objective, constraints, maxiter=100, verbose=3)

Let us again plot the $|B|$ contours in Boozer coordinates to get a qualitative picture of the solution. 
Although still not perfect, the optimized equilibrium is clearly more omnigenous compared to the initial one we plotted above. 
Now almost all of the contours are closed poloidally, except for a few "puddles" near the minimum of the field strength. 
The omnigenity could probably be further improved by using higher resolutions. 

In [ ]:
plot_boozer_surface(eq, fieldlines=4)  # defaults to the rho=1 surface

We can also plot the omnigenous field that was used as a target during the optimization, as shown below. 
This is a perfectly omnigenous magnetic field and is physically unrealistic to achieve by an equilibrium, but it represents the closest omnigenous field to the optimized solution. 
In the limit of lower omnigenity errors, this plot and the one from the equilibrium plotted above should approach becoming identical. 

Plotting the omnigenous field in Boozer coordinates requires a value for the rotational transform, so we use $\iota$ from the optimized equilibrium. 

In [ ]:
# compute the rotational transform at rho=1
grid = LinearGrid(M=eq.M_grid, N=eq.N_grid, NFP=eq.NFP, sym=eq.sym)
iota = eq.compute("iota", grid=grid)["iota"][0]

plot_omnigenous_field(field, iota, fieldlines=4)

It is also useful to compare the boundaries of the initial and optimized equilibria, as shown in this plot. 
This reveals that the optimization improved the omnigenity by adding some torsion and elongation. 

In [ ]:
plot_boundaries((eq0, eq), labels=["Initial", "Optimized"], phi=3, lw=2)